<a href="https://colab.research.google.com/github/Frederik-Roeckle/xwines_recom/blob/master/Recommender_Models/Graphs/LightGCN_XWines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

!pip install pyg-lib==0.4.0+pt26cu124 \
            torch-scatter==2.1.2+pt26cu124 \
            torch-sparse==0.6.18+pt26cu124 \
            torch-cluster==1.6.3+pt26cu124 \
            torch-spline-conv==1.2.2+pt26cu124 \
            -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

!pip install torch-geometric

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidi

In [5]:
from google.colab import drive
import torch
from torch.nn import Embedding, Linear, Module, ModuleDict
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero, MessagePassing
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.utils import structured_negative_sampling
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from tqdm import tqdm
import pathlib as pl
from sentence_transformers import SentenceTransformer
import math
import copy
import datetime
from torch_geometric.nn import LightGCN
from torch_geometric.utils import degree
import os.path as osp
from torch_geometric.utils import to_undirected, train_test_split_edges
from torch_geometric.data import Data

In [3]:
# Get PyTorch version
pytorch_version = torch.__version__
print(f"PyTorch version: {pytorch_version}")

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

# Get CUDA version (if available)
if cuda_available:
    cuda_version = torch.version.cuda
    print(f"CUDA version: {cuda_version}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4


In [22]:
csv_wines = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/XWines_Full_100K_wines.csv")
csv_training = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/trainset.csv")

csv_all_reviews = pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/XWines_Full_21M_ratings.csv")

training_reviews = pd.read_csv(csv_training)
all_reviews = pd.read_csv(csv_all_reviews)
all_wines = pd.read_csv(csv_wines)

# for shorter training time prototyping reduce trainset size
#split = 0.3
#reviews = reviews.loc[reviews.index < len(reviews) * split]

<ipython-input-22-c40c7aaaef4d>:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  all_reviews = pd.read_csv(csv_all_reviews)


In [ ]:
# learn Label encoder for user and wine on full dataset

In [43]:
user_encoder = LabelEncoder()
wine_encoder = LabelEncoder()

all_reviews['UserID_idx'] = user_encoder.fit_transform(all_reviews['UserID'])
all_reviews['WineID_idx'] = wine_encoder.fit_transform(all_reviews['WineID'])

training_reviews["UserID_idx"] = user_encoder.transform(training_reviews["UserID"])
training_reviews["WineID_idx"] = wine_encoder.transform(training_reviews["WineID"])

num_users = len(user_encoder.classes_)
num_wines = len(wine_encoder.classes_)

data = HeteroData()
data['user'].num_nodes = all_reviews['UserID'].nunique()
data['wine'].num_nodes = all_reviews['WineID'].nunique()
data['user', 'rates', 'wine'].edge_index = torch.tensor([
    training_reviews['UserID_idx'].values,
    training_reviews['WineID_idx'].values
], dtype=torch.long)

# Add ratings as edge weights
data['user', 'rates', 'wine'].edge_attr = torch.tensor(training_reviews['Rating'].values, dtype=torch.float).unsqueeze(1)
data

HeteroData(
  user={ num_nodes=1056079 },
  wine={ num_nodes=100646 },
  (user, rates, wine)={
    edge_index=[2, 16917894],
    edge_attr=[16917894, 1],
  }
)

In [47]:
# Assume data.edge_index exists and is [2, num_edges]
edge_index = data.edge_index
num_edges = edge_index.size(1)

# Shuffle indices
perm = torch.randperm(num_edges)
split = int(0.1 * num_edges)

# 10% for edge_label_index (validation/test)
edge_label_idx = perm[:split]
# 90% remain for training
train_edge_idx = perm[split:]

edge_label_index = edge_index[:, edge_label_idx]
train_edge_index = edge_index[:, train_edge_idx]

# Assign to data object
data.edge_label_index = edge_label_index
data.edge_index = train_edge_index

In [48]:
data

Data(edge_index=[2, 15226105], edge_attr=[16917894, 1], node_type=[1156725], edge_type=[16917894], edge_label_index=[2, 1691789])

In [46]:
data = data.to_homogeneous().to(device)

In [ ]:
batch_size = 8192
mask = data.edge_index[0] < data.edge_index[1]
train_edge_label_index = data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=64,
    num_layers=2,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [53]:
def train():
    total_loss = total_examples = 0

    for index in tqdm(train_loader):
        # Sample positive and negative labels.
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_wines,
                          (index.numel(), ), device=device)
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(data.edge_index, edge_label_index).chunk(2)

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples


@torch.no_grad()
def test(k: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, book_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    for start in range(0, num_users, batch_size):
        end = start + batch_size
        logits = user_emb[start:end] @ book_emb.t()

        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
               train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_label_index[0] >= start) &
                (data.edge_label_index[0] < end))
        ground_truth[data.edge_label_index[0, mask] - start,
                     data.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data.edge_label_index[0, mask] - start,
                            num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

    return precision / total_examples, recall / total_examples

In [55]:
for epoch in range(1, 2):
    loss = train()
    precision, recall = test(k=20)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Precision@20: '
          f'{precision:.4f}, Recall@20: {recall:.4f}')

100%|██████████| 1859/1859 [13:39<00:00,  2.27it/s]


Epoch: 001, Loss: 0.3973, Precision@20: 0.0024, Recall@20: 0.0214


In [56]:
torch.save(model.state_dict(), pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject") / "lightgcn_2_model.pth")
torch.save(data, pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject") / "lightgcn_2_data.pth")

In [ ]:
# Load previously stored models
data = torch.load(pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/full_3_data.pth"), weights_only=False)

encoder = GNNEncoder(hidden_channels=64)
model = to_hetero(encoder, data.metadata(), aggr='sum').to(device)#
model.load_state_dict(torch.load(pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/full_3_model.pth"), weights_only=True))

predictor = MLPPredictor(64).to(device)
predictor.load_state_dict(torch.load(pl.Path("/content/drive/MyDrive/01_Master/WebMiningProject/full_3_predictor.pth"), weights_only=True))